In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
albums = pd.read_csv('../data/new/albums_colors_nudity_skin_gender.csv')
features = pd.read_csv('../data/new/album_features_complete.csv')
artist_info = pd.read_csv('../data/new/artists_with_info_new.csv')

In [21]:
## Features cleaning ##

# Convert hex color codes to numeric RGB components
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Process each dominant color
for i in range(1, 6):
    hex_col = f'color_{i}_hex'
    r_col = f'color_{i}_R'
    g_col = f'color_{i}_G'
    b_col = f'color_{i}_B'
    
    # Apply conversion, handle missing or invalid entries
    rgb_data = features[hex_col].fillna('#000000').apply(hex_to_rgb).tolist()
    features[r_col] = [rgb[0] for rgb in rgb_data]
    features[g_col] = [rgb[1] for rgb in rgb_data]
    features[b_col] = [rgb[2] for rgb in rgb_data]

# Drop original hex columns/non-numeric/text/unneeded columns
drop_cols = [f'color_{i}_hex' for i in range(1, 6)] + ['width', 'height','aspect_ratio','image_url']
features = features.drop(columns=drop_cols)

# Convert boolean columns to integers
bool_cols = features.select_dtypes(include='bool').columns
features[bool_cols] = features[bool_cols].astype(int)

In [22]:
#Clean artist_info
artist_info = artist_info.rename(columns={'popularity':'artist_popularity'})
artist_info = artist_info.drop(['genres','artist_name'],axis=1)

#merge to albums df
albums = albums.merge(artist_info, on='artist_id')

In [24]:
df = albums.merge(features, on='album_id')
df = df.drop_duplicates()

#albums cleaning

#group gender diverse categories
group_map = {'non-binary':'gender_diverse',
    'trans woman': 'gender_diverse',
    'genderfluid':'gender_diverse'}
df['gender_grouped'] = df['gender'].replace(group_map)

# Convert boolean columns to integers
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

df.to_csv('../data/new/final_eda_df.csv')
print(df.shape)
df.head()

(3262, 101)


,artist_name,artist_id,genre,album_id,album_name,release_date,total_tracks,popularity,image_url,genre_condensed,...,color_3_R,color_3_G,color_3_B,color_4_R,color_4_G,color_4_B,color_5_R,color_5_G,color_5_B,gender_grouped
0,Taylor Swift,06HL4z0CvFAxyc27GXpf02,[],5H7ixXZfsNMGbIE5OBSpcb,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,2024-04-19,31,88,https://i.scdn.co/image/ab67616d0000b2738ecc33...,pop,...,129,124,118,88,83,78,211,206,201,female
1,Taylor Swift,06HL4z0CvFAxyc27GXpf02,[],1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,2024-04-18,16,84,https://i.scdn.co/image/ab67616d0000b2735076e4...,pop,...,163,156,146,63,55,48,201,196,188,female
2,Taylor Swift,06HL4z0CvFAxyc27GXpf02,[],1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,75,https://i.scdn.co/image/ab67616d0000b273dc2bac...,pop,...,206,204,196,107,146,175,186,135,100,female
3,Taylor Swift,06HL4z0CvFAxyc27GXpf02,[],64LU4c1nfjz1t4VnGhagcg,1989 (Taylor's Version),2023-10-26,21,84,https://i.scdn.co/image/ab67616d0000b273904445...,pop,...,109,147,176,186,135,100,207,206,198,female
4,Taylor Swift,06HL4z0CvFAxyc27GXpf02,[],5AEDGbliTTfjOB8TSm1sxt,Speak Now (Taylor's Version),2023-07-07,22,82,https://i.scdn.co/image/ab67616d0000b2730b04da...,pop,...,244,201,167,28,20,24,118,52,118,female


In [25]:
df.columns

Index(['artist_name', 'artist_id', 'genre', 'album_id', 'album_name',
       'release_date', 'total_tracks', 'popularity', 'image_url',
       'genre_condensed',
       ...
       'color_3_R', 'color_3_G', 'color_3_B', 'color_4_R', 'color_4_G',
       'color_4_B', 'color_5_R', 'color_5_G', 'color_5_B', 'gender_grouped'],
      dtype='object', length=101)